In [27]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark import SparkContext, SQLContext
import numpy as array,math

In [28]:
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(values[0], values[1:])

In [29]:
sc = SparkContext(appName="LinearRegressionWithSGD Regression")
#data = sc.textFile("C:/SparkDev/spark-1.4.0-bin-hadoop2.6/data/mllib/ridge-data/lpsa.data")


In [30]:
# Load dataset
data = sc.textFile("C:\Users\Rachan\Downloads\winequality-white_SetR.csv")
parsedData = data.map(parsePoint)


In [31]:
# 60,40 split traing, tesing data
trainingData, testData = parsedData.randomSplit([3, 2], 11L)

In [32]:
print ("Training Data Count : "+str(trainingData.count()))
print ("Test Data Count : "+str(testData.count()))

Training Data Count : 2958
Test Data Count : 1940


In [52]:
# Build the model
# regType - type of regurlarization
trainingModel = LinearRegressionWithSGD.train(trainingData,iterations=100, step=0.001, intercept=True, regType="l2")

# Evaluate the model on training data
trainingValuesAndPreds = trainingData.map(lambda p: (p.label, trainingModel.predict(p.features)))
trainingMSEMap = trainingValuesAndPreds.map(lambda (v, p): (v - p)**2)
#trainingMSEMap.collect()

In [53]:
TRMSE = math.sqrt(trainingMSEMap.reduce(lambda x, y: x + y) /  trainingValuesAndPreds.count())

print("Training Root Mean Squared Error = " + str(TRMSE))

Training Root Mean Squared Error = 1.72299463328


In [48]:
# Test on training model
testValuesAndPreds = testData.map(lambda p: (p.label, trainingModel.predict(p.features)))
testMSEMap = testValuesAndPreds.map(lambda (v, p): (v - p)**2)
#testMSEMap.collect()

In [49]:
TSTRMSE = math.sqrt(testMSEMap.reduce(lambda x, y: x + y) /  testValuesAndPreds.count())

print("Test Root Mean Squared Error = " + str(TSTRMSE))


Test Root Mean Squared Error = 1.16755891839e+41


In [25]:
sc.stop()